In [210]:
!pip install pennylane torchcam torchvision

In [211]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import train_test_split
from torchvision import transforms, datasets
import torchvision
from pennylane import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
from torchcam.methods import GradCAM, GradCAMpp
import pennylane as qml
from PIL import Image
import pandas as pd
import numpy as np
import shutil
import torch
import os
import time
import copy
from IPython.display import display
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import recall_score

In [212]:
#os.environ["OMP_NUM_THREADS"] = "1"

In [213]:
class BreastCancerDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("L")
        label = 0 if 'normal' in img_path else 1

        if self.transform:
            image = self.transform(image)

        return image, label

In [214]:
normal_dir = "/kaggle/input/thermal-breast-cancer/normal_segmented_parts"
abnormal_dir = "/kaggle/input/thermal-breast-cancer/abnormal_segmented_parts"
output_dir = "output/"
transform = transforms.Compose([transforms.ToTensor()])

In [215]:
def split_separate_folders(normal_dir, abnormal_dir, output_dir, train_ratio=0.7,
                         val_ratio=0.15, test_ratio=0.15, random_state=42):

    splits = ['train', 'val', 'test']
    classes = ['normal', 'abnormal']

    for split in splits:
        for cls in classes:
            os.makedirs(os.path.join(output_dir, split, cls), exist_ok=True)

    results = {
        'Class': [],
        'Training': [],
        'Validation': [],
        'Testing': [],
        'Total': []
    }

    for cls, src_dir in zip(classes, [normal_dir, abnormal_dir]):

        images = [f for f in os.listdir(src_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        total_images = len(images)


        train_images, temp_images = train_test_split(
            images,
            train_size=train_ratio,
            random_state=random_state
        )

        val_ratio_adjusted = val_ratio / (val_ratio + test_ratio)
        val_images, test_images = train_test_split(
            temp_images,
            train_size=val_ratio_adjusted,
            random_state=random_state
        )

        for img, split_type in zip([train_images, val_images, test_images], splits):
            for image_name in img:
                shutil.copy2(
                    os.path.join(src_dir, image_name),
                    os.path.join(output_dir, split_type, cls, image_name)
                )

        results['Class'].append(cls)
        results['Training'].append(len(train_images))
        results['Validation'].append(len(val_images))
        results['Testing'].append(len(test_images))
        results['Total'].append(total_images)

    summary_df = pd.DataFrame(results)
    display(summary_df)

    print("\nSplit Percentages:")
    for split in ['Training', 'Validation', 'Testing']:
        total = summary_df[split].sum()
        overall_total = summary_df['Total'].sum()
        print(f"{split}: {total} images ({total/overall_total*100:.1f}%)")

    return summary_df

In [216]:
summary = split_separate_folders(
    normal_dir=normal_dir,
    abnormal_dir=abnormal_dir,
    output_dir=output_dir
)

,Class,Training,Validation,Testing,Total
0,normal,350,75,75,500
1,abnormal,350,75,75,500



Split Percentages:
Training: 700 images (70.0%)
Validation: 150 images (15.0%)
Testing: 150 images (15.0%)


In [217]:
for split in ['train', 'val', 'test']:
    for cls in ['normal', 'abnormal']:
        path = os.path.join(output_dir, split, cls)
        num_images = len([f for f in os.listdir(path) if f.endswith(('.png', '.jpg', '.jpeg'))])

In [218]:
class MedicalImageDataset(Dataset):

    def __init__(self, data_dir, transform=None):

        self.data_dir = data_dir
        self.transform = transform
        self.classes = ['normal', 'abnormal']
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}

        self.images = []
        self.labels = []

        for cls in self.classes:
            class_dir = os.path.join(data_dir, cls)
            class_idx = self.class_to_idx[cls]

            for img_name in os.listdir(class_dir):
                if img_name.endswith(('.png', '.jpg', '.jpeg')):
                    self.images.append(os.path.join(class_dir, img_name))
                    self.labels.append(class_idx)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert('L')

        if self.transform:
            image = self.transform(image)

        return image, label

In [219]:
torch.manual_seed(42)

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [220]:
n_qubits = 4                
num_epochs = 40             
q_depth = 1      
batch_size=60               
q_delta = 0.01              

In [221]:
def create_data_loaders(data_dir, batch_size=60, num_workers=2):

    train_dataset = MedicalImageDataset(
        os.path.join(data_dir, 'train'),
        transform=train_transform
    )

    val_dataset = MedicalImageDataset(
        os.path.join(data_dir, 'val'),
        transform=val_test_transform
    )

    test_dataset = MedicalImageDataset(
        os.path.join(data_dir, 'test'),
        transform=val_test_transform
    )

    print(f"Training set size: {len(train_dataset)}")
    print(f"Validation set size: {len(val_dataset)}")
    print(f"Test set size: {len(test_dataset)}")

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True
    )

    return train_loader, val_loader, test_loader

In [222]:
data_dir = "output"

train_loader, val_loader, test_loader = create_data_loaders(
    data_dir=data_dir,
    batch_size=batch_size
)


for images, labels in train_loader:
    print(f"Batch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Labels: {labels}")
    break

Training set size: 700
Validation set size: 150
Test set size: 150


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch shape: torch.Size([60, 1, 224, 224])
Labels shape: torch.Size([60])
Labels: tensor([0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1,
        1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
        1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1])


In [223]:
def H_layer(nqubits):
    """Layer of single-qubit Hadamard gates.
    """
    for idx in range(nqubits):
        qml.Hadamard(wires=idx)


def RY_layer(w):
    """Layer of parametrized qubit rotations around the y axis.
    """
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)


def entangling_layer(nqubits):
    """Layer of CNOTs followed by another shifted layer of CNOT.
    """
    # In other words it should apply something like :
    # CNOT  CNOT  CNOT  CNOT...  CNOT
    #   CNOT  CNOT  CNOT...  CNOT
    for i in range(0, nqubits - 1, 2):  # Loop over even indices: i=0,2,...N-2
        qml.CNOT(wires=[i, i + 1])
    for i in range(1, nqubits - 1, 2):  # Loop over odd indices:  i=1,3,...N-3
        qml.CNOT(wires=[i, i + 1])

In [224]:
dev = qml.device("default.qubit", wires=4)


@qml.qnode(dev)
def quantum_net(q_input_features, q_weights_flat):
    """
    The variational quantum circuit.
    """

    # Reshape weights
    q_weights = q_weights_flat.reshape(q_depth, n_qubits)

    # Start from state |+> , unbiased w.r.t. |0> and |1>
    H_layer(n_qubits)

    # Embed features in the quantum node
    RY_layer(q_input_features)

    # Sequence of trainable variational layers
    for k in range(q_depth):
        entangling_layer(n_qubits)
        RY_layer(q_weights[k])

    # Expectation values in the Z basis
    exp_vals = [qml.expval(qml.PauliZ(position)) for position in range(n_qubits)]
    return tuple(exp_vals)

In [225]:
class DressedQuantumNet(nn.Module):
    """
    Torch module implementing the *dressed* quantum net.
    """

    def __init__(self):
        """
        Definition of the *dressed* layout.
        """

        super().__init__()
        self.pre_net = nn.Linear(512, n_qubits)
        self.q_params = nn.Parameter(q_delta * torch.randn(q_depth * n_qubits))
        self.post_net = nn.Linear(n_qubits, 2)

    def forward(self, input_features):
        """
        Defining how tensors are supposed to move through the *dressed* quantum
        net.
        """

        # obtain the input features for the quantum circuit
        # by reducing the feature dimension from 512 to 4
        pre_out = self.pre_net(input_features)
        q_in = torch.tanh(pre_out) * np.pi / 2.0

        # Apply the quantum circuit to each element of the batch and append to q_out
        q_out = torch.Tensor(0, n_qubits)
        q_out = q_out.to(device)
        for elem in q_in:
            q_out_elem = torch.hstack(quantum_net(elem, self.q_params)).float().unsqueeze(0)
            q_out = torch.cat((q_out, q_out_elem))

        # return the two-dimensional prediction from the postprocessing layer
        return self.post_net(q_out)

In [226]:
weights = torchvision.models.ResNet18_Weights.IMAGENET1K_V1
model_hybrid = torchvision.models.resnet18(weights=weights)

for param in model_hybrid.parameters():
    param.requires_grad = False


# Notice that model_hybrid.fc is the last layer of ResNet18
model_hybrid.fc = DressedQuantumNet()

# Use CUDA or CPU according to the "device" object.
model_hybrid = model_hybrid.to(device)

In [227]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_loss = 10000.0  # Large arbitrary number
    best_acc_train = 0.0
    best_loss_train = 10000.0  # Large arbitrary number
    print("Training started:")

    for epoch in range(num_epochs):

        # Each epoch has a training and validation phase
        for phase in ["train", "val"]:
            if phase == "train":
                # Set model to training mode
                model.train()
            else:
                # Set model to evaluate mode
                model.eval()
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = dataset_sizes[phase] // batch_size
            it = 0
            for inputs, labels in dataloaders[phase]:
                since_batch = time.time()
                batch_size_ = len(inputs)
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                # Track/compute gradient and make an optimization step only when training
                with torch.set_grad_enabled(phase == "train"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()

                # Print iteration results
                running_loss += loss.item() * batch_size_
                batch_corrects = torch.sum(preds == labels.data).item()
                running_corrects += batch_corrects
                print(
                    "Phase: {} Epoch: {}/{} Iter: {}/{} Batch time: {:.4f}".format(
                        phase,
                        epoch + 1,
                        num_epochs,
                        it + 1,
                        n_batches + 1,
                        time.time() - since_batch,
                    ),
                    end="\r",
                    flush=True,
                )
                it += 1

            # Print epoch results
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]
            print(
                "Phase: {} Epoch: {}/{} Loss: {:.4f} Acc: {:.4f}        ".format(
                    "train" if phase == "train" else "val  ",
                    epoch + 1,
                    num_epochs,
                    epoch_loss,
                    epoch_acc,
                )
            )

            # Check if this is the best model wrt previous epochs
            if phase == "val" and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == "val" and epoch_loss < best_loss:
                best_loss = epoch_loss
            if phase == "train" and epoch_acc > best_acc_train:
                best_acc_train = epoch_acc
            if phase == "train" and epoch_loss < best_loss_train:
                best_loss_train = epoch_loss

            # Update learning rate
            if phase == "train":
                scheduler.step()

    # Print final results
    model.load_state_dict(best_model_wts)
    time_elapsed = time.time() - since
    print(
        "Training completed in {:.0f}m {:.0f}s".format(time_elapsed // 60, time_elapsed % 60)
    )
    print("Best test loss: {:.4f} | Best test accuracy: {:.4f}".format(best_loss, best_acc))
    return model

In [228]:
criterion = nn.CrossEntropyLoss()
optimizer_hybrid = optim.Adam(model_hybrid.fc.parameters(), lr=0.001, weight_decay=1e-5)
exp_lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer_hybrid, step_size=10, gamma=0.1)

In [229]:
model_hybrid = train_model(
    model_hybrid, criterion, optimizer_hybrid, exp_lr_scheduler, num_epochs=num_epochs
)

Training started:
Phase: train Epoch: 1/40 Loss: 0.5574 Acc: 0.6957        
Phase: val   Epoch: 1/40 Loss: 0.3750 Acc: 0.8733        
Phase: train Epoch: 2/40 Loss: 0.4200 Acc: 0.8314        
Phase: val   Epoch: 2/40 Loss: 0.2847 Acc: 0.9200        
Phase: train Epoch: 3/40 Loss: 0.3793 Acc: 0.8486        
Phase: val   Epoch: 3/40 Loss: 0.2489 Acc: 0.9267        
Phase: train Epoch: 4/40 Loss: 0.3663 Acc: 0.8386        
Phase: val   Epoch: 4/40 Loss: 0.1963 Acc: 0.9800        
Phase: train Epoch: 5/40 Loss: 0.3370 Acc: 0.8643        
Phase: val   Epoch: 5/40 Loss: 0.1846 Acc: 0.9533        
Phase: train Epoch: 6/40 Loss: 0.3125 Acc: 0.8714        
Phase: val   Epoch: 6/40 Loss: 0.1312 Acc: 0.9800        
Phase: train Epoch: 7/40 Loss: 0.2732 Acc: 0.8914        
Phase: val   Epoch: 7/40 Loss: 0.1571 Acc: 0.9533        
Phase: train Epoch: 8/40 Loss: 0.3260 Acc: 0.8629        
Phase: val   Epoch: 8/40 Loss: 0.1149 Acc: 0.9867        
Phase: train Epoch: 9/40 Loss: 0.2625 Acc: 0.8957     

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
import cv2

class ModelWrapper(nn.Module):
    def __init__(self, model):
        super(ModelWrapper, self).__init__()
        self.model = model
        self.gradients = None
        self.activations = None
        self.gradients_squared = None
        self.activations_squared = None

        self.model.feature_extractor.conv2.register_forward_hook(self.activations_hook)
        self.model.feature_extractor.conv2.register_backward_hook(self.gradients_hook)
    
    def activations_hook(self, module, input, output):
        self.activations = output
        self.activations_squared = output ** 2
    
    def gradients_hook(self, module, grad_input, grad_output):
        self.gradients = grad_output[0]
        self.gradients_squared = grad_output[0] ** 2
    
    def forward(self, x):
        return self.model(x)
    
    def get_gradcam(self, input_image, target_class=None):
        output = self.forward(input_image)
        
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        
        self.model.zero_grad()
        
        one_hot = torch.zeros_like(output)
        one_hot[0][target_class] = 1
        output.backward(gradient=one_hot)
        
        weights = torch.mean(self.gradients, dim=(2, 3), keepdim=True)
        
        cam = torch.sum(weights * self.activations, dim=1, keepdim=True)
        cam = F.relu(cam)
        
        cam = F.interpolate(cam, size=(224, 224), mode='bilinear', align_corners=False)
        cam = cam - cam.min()
        cam = cam / (cam.max() + 1e-7)
        
        return cam.squeeze().detach().cpu().numpy(), target_class, output
    
    def get_gradcampp(self, input_image, target_class=None):
        output = self.forward(input_image)
        
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        
        self.model.zero_grad()
        
        one_hot = torch.zeros_like(output)
        one_hot[0][target_class] = 1
        output.backward(gradient=one_hot)
        
        alpha_num = self.gradients_squared
        alpha_denom = 2 * self.gradients_squared + torch.sum(
            self.activations * self.gradients, dim=(2, 3), keepdim=True
        )
        alpha_denom = torch.where(alpha_denom != 0, alpha_denom, torch.ones_like(alpha_denom))
        
        alphas = alpha_num / alpha_denom
        weights = torch.sum(alphas * F.relu(self.gradients), dim=(2, 3), keepdim=True)
        
        campp = torch.sum(weights * self.activations_squared, dim=1, keepdim=True)
        campp = F.relu(campp)
        
        campp = F.interpolate(campp, size=(224, 224), mode='bilinear', align_corners=False)
        campp = campp - campp.min()
        campp = campp / (campp.max() + 1e-7)
        
        return campp.squeeze().detach().cpu().numpy(), target_class, output
    
    def get_guided_backprop(self, input_image, target_class=None):
        def guided_hook(module, grad_in, grad_out):
            
            return (F.relu(grad_in[0]), None, None)
        
        # Register the guided hook
        self.model.feature_extractor.conv1.register_backward_hook(guided_hook)
        
        input_image.requires_grad = True
        output = self.forward(input_image)
        
        if target_class is None:
            target_class = output.argmax(dim=1).item()
        
        self.model.zero_grad()
        
        one_hot = torch.zeros_like(output)
        one_hot[0][target_class] = 1
        output.backward(gradient=one_hot)
        
        guided_gradients = input_image.grad.data
        guided_gradients = guided_gradients - guided_gradients.min()
        guided_gradients = guided_gradients / (guided_gradients.max() + 1e-7)
        
        return guided_gradients.squeeze().detach().cpu().numpy(), target_class, output
    
    def get_guided_gradcampp(self, input_image, target_class=None):
        campp, target_class, output = self.get_gradcampp(input_image, target_class)
        
        guided_gradients, _, _ = self.get_guided_backprop(input_image, target_class)
        
        guided_gradcampp = guided_gradients * campp
        
        return guided_gradcampp, target_class, output

def visualize_gradcam(image_tensor, guided_cam, guided_campp, cam, campp, target_class, output, class_names=None):
    image = image_tensor.squeeze().detach().cpu().numpy()
    
    heatmap_cam = np.uint8(255 * cam)
    heatmap_cam = cv2.applyColorMap(heatmap_cam, cv2.COLORMAP_JET)
    
    heatmap_campp = np.uint8(255 * campp)
    heatmap_campp = cv2.applyColorMap(heatmap_campp, cv2.COLORMAP_JET)
    
    probabilities = torch.nn.functional.softmax(output, dim=1)
    prob = probabilities[0][target_class].item()
    
    if class_names and target_class < len(class_names):
        class_label = f"Class: {class_names[target_class]}"
    else:
        class_label = f"Class: {target_class}"
    
    fig, axes = plt.subplots(1, 5, figsize=(20, 5))
    
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    axes[1].imshow(guided_cam, cmap='gray')
    axes[1].set_title('Guided GradCAM')
    axes[1].axis('off')
    
    axes[2].imshow(guided_campp, cmap='gray')
    axes[2].set_title('Guided GradCAM++')
    axes[2].axis('off')
    
    axes[3].imshow(cam, cmap='jet')
    axes[3].set_title(f'GradCAM\n{class_label}\nProbability: {prob:.3f}')
    axes[3].axis('off')
    
    axes[4].imshow(campp, cmap='jet')
    axes[4].set_title(f'GradCAM++\n{class_label}\nProbability: {prob:.3f}')
    axes[4].axis('off')
    
    plt.tight_layout()
    plt.show()

def compute_and_show_gradcam(model_path, image_path, target_class=None, class_names=None): 
    model = HybridModel()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    wrapped_model = ModelWrapper(model)
    
    transform = transforms.Compose([
        transforms.Grayscale(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    
    image = Image.open(image_path)
    image_tensor = transform(image).unsqueeze(0)
    image_tensor.requires_grad = True
    
    cam, pred_class, output = wrapped_model.get_gradcam(image_tensor, target_class)
    campp, pred_class_pp, output_pp = wrapped_model.get_gradcampp(image_tensor, target_class)
    guided_cam, _, _ = wrapped_model.get_guided_backprop(image_tensor, target_class)
    guided_campp, _, _ = wrapped_model.get_guided_gradcampp(image_tensor, target_class)
    
    visualize_gradcam(image_tensor, guided_cam, guided_campp, cam, campp, pred_class, output, class_names)
    
    return cam, campp, guided_cam, guided_campp, pred_class, output

In [230]:
num_epochs = 30
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model_hybrid.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model_hybrid(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = correct / total

    mode_hybrid.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_inputs, val_labels in val_loader:
            val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)

            val_outputs = model_hybrid(val_inputs)
            val_loss = criterion(val_outputs, val_labels)
            val_running_loss += val_loss.item()
            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_loss_avg = val_running_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, "
          f"Val Loss: {val_loss_avg:.4f}, Val Accuracy: {val_accuracy:.4f}")

    if val_loss_avg < best_val_loss:
        best_val_loss = val_loss_avg
        torch.save(model_hybrid.state_dict(), 'best_model.pth')

    scheduler.step()

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[60, 1, 224, 224] to have 3 channels, but got 1 channels instead

In [ ]:
def load_random_image(image_folder_path):
    image_files = [f for f in os.listdir(image_folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
    random_image_path = os.path.join(image_folder_path, random.choice(image_files))
    image = Image.open(random_image_path).convert('L')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])
    image = transform(image)
    return image

def test_random_image(model, image_folder_path):
    model.eval()

    image = load_random_image(image_folder_path)
    image = image.unsqueeze(0).to(device)
    with torch.no_grad():
              output = model(image)

    _, predicted_class = torch.max(output, 1)
    predicted_class = predicted_class.item()

    print(f"Predicted class for the random image: {'Cancer' if predicted_class == 1 else 'No Cancer'}")

image_folder_path = 'output/test/normal'
test_random_image(model_hybrid, image_folder_path)

In [ ]:
model.eval()
test_running_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    for test_inputs, test_labels in test_loader:
        test_inputs, test_labels = test_inputs.to(device), test_labels.to(device)

        test_outputs = model(test_inputs)

        test_loss = criterion(test_outputs, test_labels)

        test_running_loss += test_loss.item()
        _, test_predicted = torch.max(test_outputs.data, 1)
        test_total += test_labels.size(0)
        test_correct += (test_predicted == test_labels).sum().item()

test_loss_avg = test_running_loss / len(test_loader)
test_accuracy = test_correct / test_total

print(f"Test Loss: {test_loss_avg:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
evaluate_model(model, test_loader, device)
evaluate_model(model, val_loader, device)